# Análisis y limpieza de boletos con tarjetas compartidas

In [18]:
import pandas as pd
import datetime

In [19]:
# Primera prueba con solo los dias del tercer trimestre#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
#path= '/content/drive/My Drive/bases_mentoria/primer trimestre 2019'
path= '/content/drive/My Drive/Mentoria/tercer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()

Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,06/07/2019 12:00:07 AM,05/07/2019 11:28:07 PM,5099043,Cor 1 Naranja,L12,Ida
1,06/07/2019 12:00:29 AM,05/07/2019 11:32:24 PM,5515754,Cor 6 Naranja,L67,Ida
2,06/07/2019 12:00:29 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
3,06/07/2019 12:00:30 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
4,06/07/2019 12:00:33 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta


In [20]:
big_frame.shape

(12718216, 6)

# Problema 1
Si la base está en un solo archivo (al menos para la misma línea y sentido) no surge el problema que se trata a continuación.

Si los datos están en archivos diarios hay que tener en cuenta que un servicio que arrancó cerca de la media noche puede tener boletos en dos días. A continuación se muestra cómo agrupar a los boletos según fecha de apertura (en vez de fecha del boleto). 

Esto es necesario para contar 1 sola vez a cada servicio y para considerar correctamente si hay tarjetas compartidas.


In [21]:
big_frame['fecha_apertura2'] = pd.to_datetime(big_frame['FECHAAPERTURA'], format='%d/%m/%Y %I:%M:%S %p')

In [22]:
big_frame.fecha_apertura2.value_counts()

2019-08-06 07:23:12    309
2019-07-08 16:51:02    236
2019-08-07 06:42:04    223
2019-07-26 16:35:22    212
2019-09-06 17:04:15    207
                      ... 
2019-08-30 14:06:34      1
2019-08-30 14:50:40      1
2019-08-30 14:23:05      1
2019-07-19 09:58:03      1
2019-09-26 23:55:31      1
Name: fecha_apertura2, Length: 350611, dtype: int64

# Problema 2

El objetivo es analizar qué % boletos son compartidos. 

Para un posterior análisis se mantendrá en la base solo el primer uso de la tarjeta en cada viaje.

In [23]:
# 1. Se depura la base, nos quedamos solo con el primer uso de la tarjeta en ese mismo viaje.
df1 = big_frame.groupby(['TARJETA','LINEA','SENTIDO','fecha_apertura2']).first().reset_index()

In [24]:
df1.head()

,TARJETA,LINEA,SENTIDO,fecha_apertura2,FECHA,FECHAAPERTURA,CORREDOR
0,2016806,A,Vuelta,2019-07-15 09:22:06,15/07/2019 10:07:20 AM,15/07/2019 09:22:06 AM,TROLE
1,2016806,L11,Ida,2019-08-05 13:43:09,05/08/2019 02:41:03 PM,05/08/2019 01:43:09 PM,Cor 1 Naranja
2,2016806,L11,Ida,2019-09-16 13:44:10,16/09/2019 02:41:45 PM,16/09/2019 01:44:10 PM,Cor 1 Naranja
3,2016806,L30,Ida,2019-08-30 06:44:45,30/08/2019 07:18:02 AM,30/08/2019 06:44:45 AM,Cor 3 Rojo
4,2016806,L30,Ida,2019-09-04 06:45:03,04/09/2019 07:17:22 AM,04/09/2019 06:45:03 AM,Cor 3 Rojo


In [25]:
n1 = big_frame.shape[0]
n2 = df1.shape[0]

n1,n2, 100*n2/n1 
## Se observa que 8% de los boletos se comparten**

(12718216, 11859700, 93.24971363908271)

In [26]:
# Tarea 1: Analizar si la proporción de boletos compartidos es diferente por linea y por hora.
## 1.1 boletos compartidos por línea##
## recordar que estamos viendo solo primer trimestre 2019
big_frame['CANT_VIAJES_TOT']=1
df_total_viajes = big_frame.groupby('LINEA')['CANT_VIAJES_TOT'].sum()
df_total_viajes= df_total_viajes.sort_values(ascending=False)

In [27]:
df_total_viajes.head(15)

LINEA
L60    468168
L70    439132
L10    365762
L12    358342
L50    349440
L40    335823
L14    332035
L32    328846
L30    326531
L11    321516
L62    315893
L65    306513
B      276628
L36    273584
L82    272056
Name: CANT_VIAJES_TOT, dtype: int64

In [28]:
## LO MISMO PERO DESCONTADOS LOS BOLETOS PRESTADOS##
df1['CANT_VIAJES_S_DUPL']=1
df_total_viajes_2 = df1.groupby('LINEA')['CANT_VIAJES_S_DUPL'].sum()
df_total_viajes_2= df_total_viajes_2.sort_values(ascending=False)




In [29]:
df_total_viajes_2.head(15)

LINEA
L60    437060
L70    413510
L10    339185
L12    332847
L50    323145
L40    313603
L32    307721
L30    306850
L14    303874
L11    298229
L62    293468
L65    285477
B      260128
L82    254912
L36    254401
Name: CANT_VIAJES_S_DUPL, dtype: int64

In [30]:
df_unido= pd.merge(df_total_viajes_2, df_total_viajes, on="LINEA")

In [31]:
df_unido.head(15)

,CANT_VIAJES_S_DUPL,CANT_VIAJES_TOT
LINEA,,
L60,437060,468168
L70,413510,439132
L10,339185,365762
L12,332847,358342
L50,323145,349440
L40,313603,335823
L32,307721,328846
L30,306850,326531
L14,303874,332035


In [32]:

df_unido['Porc_tarj_prestadas']= 1 -(df_unido.CANT_VIAJES_S_DUPL/df_unido.CANT_VIAJES_TOT)


In [33]:

df_unido.head(15)

,CANT_VIAJES_S_DUPL,CANT_VIAJES_TOT,Porc_tarj_prestadas
LINEA,,,
L60,437060,468168,0.066446
L70,413510,439132,0.058347
L10,339185,365762,0.072662
L12,332847,358342,0.071147
L50,323145,349440,0.075249
L40,313603,335823,0.066166
L32,307721,328846,0.064240
L30,306850,326531,0.060273
L14,303874,332035,0.084813


In [34]:
#exportamos de a un trimestre las bases colapsadas, en este caso a nivel linea de los boletos totales y los boletos sin contar los prestados3
df_unido= df_unido.sort_values(by=['Porc_tarj_prestadas'],ascending=False)
df_unido.head(10)
viajes_prestados_3t19= df_unido
viajes_prestados_3t19['LINEA']=viajes_prestados_3t19.index
viajes_prestados_3t19.to_csv("viajes_prestados_3t19.csv", index=None)
from google.colab import files
files.download('viajes_prestados_3t19.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>